<a href="https://colab.research.google.com/github/finardi/Ranking/blob/main/0_dataprep_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -q rank_bm25

In [ ]:
import string
import pickle
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from sklearn.utils import shuffle
from tqdm.autonotebook import tqdm
from collections import OrderedDict

In [ ]:
def pickle_file(path, data=None):
    if data is None:
        with open(path, 'rb') as f:
            return pickle.load(f)
    if data is not None:
        with open(path, 'wb') as handle:
            pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# reading RECEITA FEDERAL FAQ
corpus =  pickle_file('/content/drive/MyDrive/Ranking - Tutos/receita_faq_list')

# Shuffle no corpus
corpus = shuffle(corpus)

def make_qa(corpus):
    questions, docs = [],[]
    for text in corpus:
        q_point = text.find('?')+1
        questions.append(text[:q_point].strip())
        docs.append(text[q_point:].strip())
    assert len(questions) == len(docs), f'ERrooou'

    return questions, docs

q, d = make_qa(corpus)

df = pd.DataFrame(
    {
        'query': q,
        'doc': d,
    }
    )
df

,query,doc
0,como deve declarar o contribuinte viúvo no dec...,"no curso do inventário, apresenta declaração c..."
1,como deve ser preenchida a declaração de bens ...,na declaração de bens e direitos correspondent...
2,como são tributados os valores pagos pelas ent...,no caso de contribuintes não optantes pelo reg...
3,incide o imposto sobre a renda sobre os rendim...,sim. os juros decorrentes de empréstimos conce...
4,qual é o tratamento tributário dos ganhos em o...,quando as operações forem realizadas em bolsas...
...,...,...
674,limite global para a dedução de despesas com i...,sim. não se enquadram no conceito de despesas ...
675,as despesas de custeio escrituradas no livro-c...,profissional autônomo pode escriturar o livro-...
676,quando devem ser tributados os rendimentos (co...,os rendimentos são tributados no mês em que fo...
677,resultado negativo ou perda apurado em um mês ...,não se pode compensar resultados negativos de ...


# split data

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

valid_size = int(df.shape[0] * 0.1)

df_train = df[valid_size:].reset_index(drop=True)
df_valid = df[:valid_size].reset_index(drop=True)
print(df.shape, df_train.shape, df_valid.shape)
df_train

(679, 2) (612, 2) (67, 2)


,query,doc
0,os valores recebidos em razão do encargo estip...,sim. doação modal ou onerosa é aquela que tra...
1,contribuinte que optar pelo desconto simplific...,"sim. contribuinte, independentemente da opção..."
2,como deve proceder o contribuinte que perdeu o...,contribuinte pode solicitar confirmação do pag...
3,é tributável a pensão alimentícia judicial ou ...,não. os valores recebidos a título de pensão e...
4,qual é o tratamento tributário da complementaç...,é isenta do imposto sobre a renda a complement...
...,...,...
607,imposto apurado na declaração de ajuste anual ...,saldo do imposto pode ser pago em até 8 (oito)...
608,"que se compreende no conceito de ""diárias"" e d...","diárias: conceituam-se diárias, para esse efei..."
609,"os gastos com medicamentos, inclusive vacinas,...","não, a não ser que integrem a conta emitida pe..."
610,qual é o tratamento tributário dos rendimentos...,no sistema de locação conjunta de unidades imo...


In [ ]:
# ✨ TRAIN_prep
pid_to_doc_TRAIN = {k:v for k, v in enumerate(df_train.doc.unique())}
doc_to_pid_TRAIN = {k:v for v, k in pid_to_doc_TRAIN.items()}
qid_to_query_TRAIN = {k:v for k, v in enumerate(df_train['query'].unique())}
query_to_qid_TRAIN = {k:v for v, k in qid_to_query_TRAIN.items()}

assert len(pid_to_doc_TRAIN) == df_train.doc.unique().shape[0] == len(doc_to_pid_TRAIN)
assert len(qid_to_query_TRAIN) == df_train['query'].unique().shape[0] == len(query_to_qid_TRAIN)

df_train = df_train.assign(qid = [query_to_qid_TRAIN[v] for v in df_train['query'].values])
df_train = df_train.assign(pid = [doc_to_pid_TRAIN[v]   for v in df_train.doc.values])
df_train = df_train[['qid', 'query', 'pid', 'doc']]

# ✨ VALID_prep
pid_to_doc_VALID = {k:v for k, v in enumerate(df_valid.doc.unique())}
doc_to_pid_VALID = {k:v for v, k in pid_to_doc_VALID.items()}
qid_to_query_VALID = {k:v for k, v in enumerate(df_valid['query'].unique())}
query_to_qid_VALID = {k:v for v, k in qid_to_query_VALID.items()}

assert len(pid_to_doc_VALID)   == df_valid.doc.unique().shape[0] == len(doc_to_pid_VALID)
assert len(qid_to_query_VALID) == df_valid['query'].unique().shape[0] == len(query_to_qid_VALID)

df_valid = df_valid.assign(qid = [query_to_qid_VALID[v] for v in df_valid['query'].values])
df_valid = df_valid.assign(pid = [doc_to_pid_VALID[v]   for v in df_valid.doc.values])
df_valid = df_valid[['qid', 'query', 'pid', 'doc']]

# MS MARCO SHAPE

In [ ]:
# =================
# ✨ make_triplets
# =================
def make_triplets(qrels, pid_to_doc):
    # triplets: qid pos_pid neg_pid
    qid_list, pos_pid_list, neg_pid_list = [], [], []
    for qid, pos_pid in zip(qrels.qid.values, qrels.pid.values):
        for positive in pos_pid:
            qid_list.append(qid)
            pos_pid_list.append(positive)
            neg_pid = np.random.choice(list(pid_to_doc.keys()))
            while neg_pid in pos_pid:
                neg_pid = np.random.choice(list(pid_to_doc.keys()))
            neg_pid_list.append(neg_pid)

    len(qid_list), len(pos_pid_list), len(neg_pid_list)      

    df_triplet = pd.DataFrame({'qid': qid_list, 'pos_pid': pos_pid_list, 'neg_pid': neg_pid_list})
    return df_triplet

# - - - - -
TRIPLET_qrels_train = df_train.groupby(['qid']).agg(
    lambda x: tuple(x)).applymap(list).reset_index()[['qid', 'pid']]

# dataframe qid pid
TRIPLET_qrels_valid = df_valid.groupby(['qid']).agg(
    lambda x: tuple(x)).applymap(list).reset_index()[['qid', 'pid']]

df_triplet_train = make_triplets(TRIPLET_qrels_train, pid_to_doc_TRAIN)
df_triplet_valid = make_triplets(TRIPLET_qrels_valid, pid_to_doc_VALID)

print(df_triplet_train.shape, df_triplet_valid.shape)    

(612, 3) (67, 3)


In [ ]:
# ✨ FAQ QUERIES MS MARCO SHAPE
faq_queries_train = OrderedDict({qid:question for qid, question in zip(df_train['qid'].values, df_train['query'].values)})
faq_queries_valid = OrderedDict({qid:question for qid, question in zip(df_valid['qid'].values, df_valid['query'].values)})

In [ ]:
# ✨ FAQ QRELS MS MARCO SHAPE
faq_qrels_train = OrderedDict({qid:pid for qid, pid in zip(TRIPLET_qrels_train['qid'].values, TRIPLET_qrels_train['pid'].values)})
faq_qrels_valid = OrderedDict({qid:pid for qid, pid in zip(TRIPLET_qrels_valid['qid'].values, TRIPLET_qrels_valid['pid'].values)})

## TopK retrieval

In [ ]:
class Metrics:
    def __init__(self, mrr_depths:set, recall_depths:set, success_depths:set, total_queries=None):
        self.results = {}
        self.mrr_sums = {depth:0.0 for depth in mrr_depths}
        self.recall_sums = {depth:0.0 for depth in recall_depths}
        self.success_sums = {depth:0.0 for depth in success_depths}
        self.total_queries = total_queries

    def get_result(self, query_idx, query_key, ranking, gold_positives):
        assert query_key not in self.results
        assert len(self.results) <= query_idx
        assert len(set(gold_positives)) == len(gold_positives)
        assert len(set([pid for _, pid, _ in ranking])) == len(ranking)

        self.results[query_key] = ranking

        positives = [i for i, (_, pid, _) in enumerate(ranking) if pid in gold_positives]

        if len(positives) == 0:
            return

        for depth in self.mrr_sums:
            first_positive = positives[0]
            self.mrr_sums[depth] += (1.0 / (first_positive+1.0)) if first_positive < depth else 0.0

        for depth in self.success_sums:
            first_positive = positives[0]
            self.success_sums[depth] += 1.0 if first_positive < depth else 0.0

        for depth in self.recall_sums:
            num_positives_up_to_depth = len([pos for pos in positives if pos < depth])
            self.recall_sums[depth] += num_positives_up_to_depth / len(gold_positives)

    def print_metrics(self, query_idx):
        print('- '*10)
        for depth in sorted(self.mrr_sums):
            mrr_value =  self.mrr_sums[depth] / (query_idx+1.0)
            print(f"MRR@{str(depth):<2} = {mrr_value:.3}")
        
        print('- '*10)
        for depth in sorted(self.recall_sums):
            recall_value = self.recall_sums[depth] / (query_idx+1.0)
            print(f"Recall@{str(depth):<2} = {recall_value:.3}")
        print('- '*10)
        for depth in sorted(self.success_sums):
            success_value = self.success_sums[depth] / (query_idx+1.0)
            print(f"Success@{str(depth):<2} = {success_value:.3}")
        print('- '*10)

# ----------------------------------------------------------------------
def pt_stop_words(path):
    # read_file with br stop_words
    with open(path) as f:
        stop_words = f.readlines()

    pt_stop_words = []
    for w in stop_words:
        # remove break lines and spaces
        pt_stop_words.append(w.replace('\n', '').strip())

    return pt_stop_words
# - - - - - 
path = '/content/drive/MyDrive/Colab Notebooks/BRstopwords.txt'
stop_words = pt_stop_words(path)

# ----------------------------------------------------------------------
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in stop_words:
            tokenized_doc.append(token)
    return tokenized_doc

# ----------------------------------------------------------------------
def search_all(metrics, queries=None, K=None, qrels=None, pid_to_doc=None):
    tokenized_corpus = []
    for passage in tqdm(pid_to_doc.values()):
        tokenized_corpus.append(bm25_tokenizer(passage))
    bm25 = BM25Okapi(tokenized_corpus)
      
    topK_pids = {}
    
    keys = sorted(list(queries.keys()))
    for query_idx, qid in enumerate(keys):
        query = queries[qid]
        
        ##### ✨ BM25 search (lexical search) #####
        bm25_scores = bm25.get_scores(bm25_tokenizer(query))
        top_n = np.argpartition(bm25_scores, -5)[-K:]
        bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
        bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
        
        ranked_scores, ranked_pids, ranked_passages = [],[],[]
        for hit in bm25_hits[0:K]:
            ranked_scores.append(hit['score'])
            ranked_pids.append(hit['corpus_id'])
            ranked_passages.append(pid_to_doc[hit['corpus_id']])
        
        topK_pids[qid] = ranked_pids
        
        ranking = list(zip(ranked_scores, ranked_pids, ranked_passages))
        
        if qrels:
            metrics.get_result(query_idx, qid, ranking, qrels[qid])
            if query_idx%25 == 0:
                print(f'\n[{query_idx}]. Query: {query}')
                for i, (score, pid, passage) in enumerate(ranking, 1):
                    if pid in qrels[qid]:
                        print(f"Found at position: {i} with score {score:.3}")
                        print(passage)ahh, foi mais legal
                        break
                metrics.print_metrics(query_idx)
            
    return topK_pids

In [ ]:
# ✨ TRAIN Retrieval test
metrics = Metrics(
        mrr_depths={1, 3, 5, 10, 20}, 
        recall_depths={1, 3, 5, 10, 20},
        success_depths={1, 3, 5, 10, 20},
        total_queries=len(faq_queries_train)
        )

topK_pids_train = search_all(
    metrics=metrics, 
    queries=faq_queries_train, 
    K=50, 
    qrels=faq_qrels_train,
    pid_to_doc=pid_to_doc_TRAIN,
    ) 



[0]. Query: os valores recebidos em razão do encargo estipulado em doação modal de bens ou direitos  são tributáveis?
Found at position: 1 with score 26.5
sim.  doação modal ou onerosa é aquela que traz consigo um encargo para o donatário. os valores recebidos  em função desse encargo estão sujeitos ao recolhimento mensal (carnê-leão), se recebidos de pessoa física  ou, na fonte, se pagos por pessoa jurídica, e na declaração de ajuste.
- - - - - - - - - - 
MRR@1  = 1.0
MRR@3  = 1.0
MRR@5  = 1.0
MRR@10 = 1.0
MRR@20 = 1.0
- - - - - - - - - - 
Recall@1  = 1.0
Recall@3  = 1.0
Recall@5  = 1.0
Recall@10 = 1.0
Recall@20 = 1.0
- - - - - - - - - - 
Success@1  = 1.0
Success@3  = 1.0
Success@5  = 1.0
Success@10 = 1.0
Success@20 = 1.0
- - - - - - - - - - 

[25]. Query: é devido imposto sobre a renda de contribuinte que faleceu após a apresentação da  declaração do exercício?
- - - - - - - - - - 
MRR@1  = 0.5
MRR@3  = 0.609
MRR@5  = 0.609
MRR@10 = 0.614
MRR@20 = 0.617
- - - - - - - - - - 
Recall@

In [ ]:
# ✨ VALID retrieval test
metrics = Metrics(
        mrr_depths={1, 3, 5, 10, 20}, 
        recall_depths={1, 3, 5, 10, 20},
        success_depths={1, 3, 5, 10, 20},
        total_queries=len(faq_queries_valid)
        )

topK_pids_valid = search_all(
    metrics=metrics,
    queries=faq_queries_valid, 
    K=50, 
    qrels=faq_qrels_valid,
    pid_to_doc=pid_to_doc_VALID,
    ) 



[0]. Query: como se distinguem os contratos agrários?
Found at position: 1 with score 2.67
os contratos de arrendamento e parceria são basicamente semelhantes no que concerne à natureza jurídica,  pois em todos há cessão de uso e gozo de imóvel ou de área rural, parte ou partes dos mesmos, incluindo,  ou não, outros bens, benfeitorias e facilidades, para ser exercida atividade de exploração agrícola, pecuária,  agroindustrial, extrativa vegetal ou mista. diferem, porém, substancialmente na forma de remuneração do  cedente:   a) no arrendamento ou subarrendamento, o cedente (arrendador ou subarrendador) recebe do arrendatário  ou subarrendatário retribuição certa ou aluguel pelo uso dos bens cedidos (os rendimentos devem ser  tributados como aluguéis, separados da atividade rural);   b) na parceria ou subparceria, o cedente (parceiro-outorgante) partilha com o parceiro-outorgado os riscos de  caso fortuito e força maior, os frutos, produtos ou lucros havidos, nas proporções estipulada

In [ ]:
def build_topk_docs(topK_pids, qid_to_query, pid_to_doc):
    queries = OrderedDict()
    topK_docs = OrderedDict()

    for i, (qid, pids) in enumerate(topK_pids.items()):
        queries[qid] = qid_to_query[qid]
        topK_docs[qid] = topK_docs.get(qid, [])

        for j, pid in enumerate(pids, 1):
            topK_docs[qid].append(pid_to_doc[pid])
    return queries, topK_docs

queries_train, topK_docs_train = build_topk_docs(
    topK_pids_train, 
    qid_to_query_TRAIN, 
    pid_to_doc_TRAIN
    )
queries_valid, topK_docs_valid = build_topk_docs(
    topK_pids_valid, 
    qid_to_query_VALID,
    pid_to_doc_VALID
    )

print('TRAIN OBJECTS')
assert len(queries_train) == len(topK_docs_train) == len(topK_pids_train)
print(f'\tlen(queries_train):    {len(queries_train)}')
print(f'\tlen(topK_docs_train):  {len(topK_docs_train)}')
print(f'\tlen(topK_pids_train):  {len(topK_pids_train)}')
print(f'\tlen(collection_train): {len(pid_to_doc_TRAIN)}')

print('\nVALID OBJECTS')
assert len(queries_valid) == len(topK_docs_valid) == len(topK_pids_valid)
print(f'\tlen(queries_valid):    {len(queries_valid)}')
print(f'\tlen(topK_docs_valid):  {len(topK_docs_valid)}')
print(f'\tlen(topK_pids_valid):  {len(topK_pids_valid)}')
print(f'\tlen(collection_valid): {len(pid_to_doc_VALID)}')

# topK_pids qid: [pids]        --> dict qid: pids list (BM25's rank)
# topK_docs : qid: [text_docs] --> dict qid: docs list (BM25's rank)
# queries: qid: text           --> dict qid: query text
# qrels: qid: [pids]           --> dict qid: relevants pids list to qid
# collection pid: passage      --> dict pid: text passage
# triples                      --> dataframe: qid | pos_pid | neg_pid

TRAIN OBJECTS
	len(queries_train):    612
	len(topK_docs_train):  612
	len(topK_pids_train):  612
	len(collection_train): 612

VALID OBJECTS
	len(queries_valid):    67
	len(topK_docs_valid):  67
	len(topK_pids_valid):  67
	len(collection_valid): 67


# Saving and Test

In [ ]:
def pickle_file(path, data=None):
    if data is None:
        with open(path, 'rb') as f:
            return pickle.load(f)
    if data is not None:
        with open(path, 'wb') as handle:
            pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

#===========
# ✨ Saving
#===========

SAVE = True
path_base = '/content/drive/MyDrive/ColBERT/ColBERT - FAQ Receita Federal/'

if SAVE:

    # DATAFRAME
    df.to_parquet(path_base+'data/df_FAQ_processed.parquet.gzip', compression='gzip')
    df_train.to_parquet(path_base+'data/df_FAQ_TRAIN.parquet.gzip', compression='gzip')
    df_valid.to_parquet(path_base+'data/df_FAQ_VALID.parquet.gzip', compression='gzip')
    
    df_triplet_train.to_parquet(path_base+'data/df_FAQ_triplet_IDS_TRAIN.parquet.gzip', compression='gzip')
    df_triplet_valid.to_parquet(path_base+'data/df_FAQ_triplet_IDS_VALID.parquet.gzip', compression='gzip')

    # DICTS
    pickle_file(path_base+'data/query_to_qid_TRAIN' , query_to_qid_TRAIN)
    pickle_file(path_base+'data/qid_to_query_TRAIN' , qid_to_query_TRAIN)
    pickle_file(path_base+'data/doc_to_pid_TRAIN'   , doc_to_pid_TRAIN)
    pickle_file(path_base+'data/pid_to_doc_TRAIN'   , pid_to_doc_TRAIN)
    
    #------------------------------------------------------------------------
    pickle_file(path_base+'data/query_to_qid_VALID' , query_to_qid_VALID)
    pickle_file(path_base+'data/qid_to_query_VALID' , qid_to_query_VALID)
    pickle_file(path_base+'data/doc_to_pid_VALID'   , doc_to_pid_VALID)
    pickle_file(path_base+'data/pid_to_doc_VALID'   , pid_to_doc_VALID)

    # MSMARCO
    pickle_file(path_base+'data/topK_pids_TRAIN' , topK_pids_train)
    pickle_file(path_base+'data/topK_docs_TRAIN' , topK_docs_train)
    pickle_file(path_base+'data/queries_TRAIN'   , queries_train)
    pickle_file(path_base+'data/qrels_TRAIN'     , faq_qrels_train)
    pickle_file(path_base+'data/collection_TRAIN', pid_to_doc_TRAIN)
    #---------------------------------------------------------
    pickle_file(path_base+'data/topK_pids_VALID' , topK_pids_valid)
    pickle_file(path_base+'data/topK_docs_VALID' , topK_docs_valid)
    pickle_file(path_base+'data/queries_VALID'   , queries_valid)
    pickle_file(path_base+'data/qrels_VALID'     , faq_qrels_valid)
    pickle_file(path_base+'data/collection_VALID', pid_to_doc_VALID)

In [ ]:
# ================
# ✨ Loading test
# ================

# DATAFRAME
df = pd.read_parquet(path_base+'data/df_FAQ_processed.parquet.gzip')
print(f'unique docs:      {df["doc"].nunique()}')
print(f'unique questions: {df["query"].nunique()}')

df_triplet_train = pd.read_parquet(path_base+'data/df_FAQ_triplet_IDS_TRAIN.parquet.gzip')
df_triplet_valid = pd.read_parquet(path_base+'data/df_FAQ_triplet_IDS_VALID.parquet.gzip')

# DICTS
query_to_qid_TRAIN = pickle_file(path_base+'data/query_to_qid_TRAIN' )
qid_to_query_TRAIN = pickle_file(path_base+'data/qid_to_query_TRAIN')
doc_to_pid_TRAIN   = pickle_file(path_base+'data/doc_to_pid_TRAIN' )
pid_to_doc_TRAIN   = pickle_file(path_base+'data/pid_to_doc_TRAIN')
#-----------------------------------------------------------------
query_to_qid_VALID = pickle_file(path_base+'data/query_to_qid_VALID')
qid_to_query_VALID = pickle_file(path_base+'data/qid_to_query_VALID')
doc_to_pid_VALID   = pickle_file(path_base+'data/doc_to_pid_VALID')
pid_to_doc_VALID   = pickle_file(path_base+'data/pid_to_doc_VALID')

# MSMARCO
topK_pids_train  = pickle_file(path_base+'data/topK_pids_TRAIN')
topK_docs_train  = pickle_file(path_base+'data/topK_docs_TRAIN')
queries_train    = pickle_file(path_base+'data/queries_TRAIN')
qrels_train      = pickle_file(path_base+'data/qrels_TRAIN')
collection_train = pickle_file(path_base+'data/collection_TRAIN')

topK_pids_valid  = pickle_file(path_base+'data/topK_pids_VALID')
topK_docs_valid  = pickle_file(path_base+'data/topK_docs_VALID')
queries_valid    = pickle_file(path_base+'data/queries_VALID')
qrels_valid      = pickle_file(path_base+'data/qrels_VALID')
collection_valid = pickle_file(path_base+'data/collection_VALID')

unique docs:      679
unique questions: 679
